# Moneyball

## Normal
- Players with the highest OBP and the lowest salary in any specific year.
![Image of OPB](https://wikimedia.org/api/rest_v1/media/math/render/svg/d497b106820c509fa8baa93f3c42cc5087fb0fea)
- Make sure you are removing outliers (an OBP of 1.0 is not an indicator of a perfect player, more like they possibly only played 4 or 5 games and had good luck, alternatively an OBP of 0 is pretty bad). 
- Your 9 player roster will need to include one player at each position.  
  - A player that historically played multiple positions can not account for 2 places on your roster.



## Advanced Mode
- Adjust each player's salary for inflation based on the year they earned the salary.
- Also you are required to put together an "all time best" legacy team based on their adjusted salary.

#### Notes
Ken Griffey Jr. 'griffke02'

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
%matplotlib inline

In [2]:
master = pd.read_csv('data/Master.csv', usecols=['playerID', 'nameFirst', 'nameLast']).dropna()
salaries = pd.read_csv('data/Salaries.csv').dropna()
batting = pd.read_csv('data/Batting.csv', usecols=['yearID', 'playerID', 'AB', 'H', 'BB', 'HBP', 'SF']).dropna()
appearances = pd.read_csv('data/Appearances.csv',
                          usecols=['yearID', 'playerID', 'G_p', 'G_c', 'G_1b', 
                                   'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_all', 'GS']).dropna()

## Data Cleanup

#### Appearances: 
To account for traded players, I want to group by player id and year, adding totals for the following categories:
- G_p            Games as pitcher
- G_c            Games as catcher
- G_1b           Games as firstbaseman
- G_2b           Games as secondbaseman
- G_3b           Games as thirdbaseman
- G_ss           Games as shortstop
- G_lf           Games as leftfielder
- G_cf           Games as centerfielder
- G_rf           Games as right fielder

To meet the assignment definition ("(Left, Center, Right) Fielders") we will exclude:
- G_of           Games as outfielder

Final Data will include position fields mentioned above and following fields:
- yearID         Year
- playerID       Player ID code
- G_all          Total games played
- GS             Games started


In [3]:
#player_stats = appearances.merge(batting, on=['playerID','yearID']).sort_values(by=['playerID', 'yearID'])
#player_stats.loc[player_stats['playerID'] == 'griffke02']

In [4]:
#first, collapse rows with same yearID and playerID, other columns should be sums
#a = appearances.loc[appearances['playerID'] == 'griffke02']
yearly_player_app = appearances.groupby(['playerID','yearID']).sum()
yearly_player_bat = batting.groupby(['playerID','yearID']).sum()
yearly_player_bat.head()


AB    H   BB  HBP   SF
playerID  yearID                         
aardsda01 2004    0.0  0.0  0.0  0.0  0.0
          2006    2.0  0.0  0.0  0.0  0.0
          2007    0.0  0.0  0.0  0.0  0.0
          2008    1.0  0.0  0.0  0.0  0.0
          2009    0.0  0.0  0.0  0.0  0.0

In [5]:
player_stats = yearly_player_app.merge(yearly_player_bat, left_index=True, right_index=True)
player_stats['OBP'] = ((player_stats['H'] + player_stats['BB'] + player_stats['HBP']) / 
                        (player_stats['AB'] + player_stats['BB'] + player_stats['HBP'] + player_stats['SF']))
player_stats = player_stats.dropna()
player_stats.head()

G_all     GS  G_p  G_c  G_1b  G_2b  G_3b  G_ss  G_lf  G_cf  \
playerID  yearID                                                               
aardsda01 2006     45.0    0.0   45    0     0     0     0     0     0     0   
          2008     47.0    0.0   47    0     0     0     0     0     0     0   
          2015     33.0    0.0   33    0     0     0     0     0     0     0   
aaronha01 1974    112.0   89.0    0    0     0     0     0     0    89     0   
          1975    137.0  129.0    0    0     0     0     0     0     3     0   

                  G_rf     AB      H    BB  HBP   SF       OBP  
playerID  yearID                                                
aardsda01 2006       0    2.0    0.0   0.0  0.0  0.0  0.000000  
          2008       0    1.0    0.0   0.0  0.0  0.0  0.000000  
          2015       0    1.0    0.0   0.0  0.0  0.0  0.000000  
aaronha01 1974       0  340.0   91.0  39.0  0.0  2.0  0.341207  
          1975       0  465.0  109.0  70.0  1.0  6.0  0.332103

In [6]:
#Merge Master and Salaries
player_info = master.merge(salaries, on='playerID')
player_info.head()

,playerID,nameFirst,nameLast,yearID,teamID,lgID,salary
0,aardsda01,David,Aardsma,2004,SFN,NL,300000
1,aardsda01,David,Aardsma,2007,CHA,AL,387500
2,aardsda01,David,Aardsma,2008,BOS,AL,403250
3,aardsda01,David,Aardsma,2009,SEA,AL,419000
4,aardsda01,David,Aardsma,2010,SEA,AL,2750000


In [7]:
player_info = player_info.groupby(['playerID', 'yearID']).sum()
player_info.head()

salary
playerID  yearID         
aardsda01 2004     300000
          2007     387500
          2008     403250
          2009     419000
          2010    2750000

In [8]:
#Add inflation table
bls_cpi = pd.read_csv('bls_cpi.csv', index_col=0)
#bls_cpi.rename(index={0: 'a'})
bls_cpi.index.names = ['yearID']
bls_cpi.tail()

,CPI
yearID,
2012,229.593273
2013,232.948727
2014,236.911091
2015,237.061727
2016,238.782333


In [9]:
#Merge CPI with salary
player_info = player_info.join(bls_cpi)
player_info.head()

salary         CPI
playerID  yearID                     
aardsda01 2004     300000  188.754546
          2007     387500  207.097545
          2008     403250  215.763818
          2009     419000  214.408636
          2010    2750000  217.953364

In [10]:
#Add an Adjusted for 2016 salary column
cpi_2016 = bls_cpi.loc[2016].values[0]

player_info['2016 Salary'] =  (player_info['salary'] * (cpi_2016 / player_info['CPI']))
player_info.head()

salary         CPI   2016 Salary
playerID  yearID                                   
aardsda01 2004     300000  188.754546  3.795124e+05
          2007     387500  207.097545  4.467854e+05
          2008     403250  215.763818  4.462703e+05
          2009     419000  214.408636  4.666314e+05
          2010    2750000  217.953364  3.012807e+06

In [11]:
#Our Final Merge
big_data = player_info.merge(player_stats, left_index=True, right_index=True).dropna()
#Now add a column for Salary/OBP.  We will use this metric to rank our players (lower is better).
big_data['2016 Salary/OBP'] = (big_data['2016 Salary'] / big_data['OBP'])
big_data = big_data.sort_values(by='2016 Salary/OBP')
big_data.head()

,,salary,CPI,2016 Salary,G_all,GS,G_p,G_c,G_1b,G_2b,G_3b,...,G_lf,G_cf,G_rf,AB,H,BB,HBP,SF,OBP,2016 Salary/OBP
playerID,yearID,,,,,,,,,,,,,,,,,,,,,
jamesdi01,1993,0,144.336364,0.000000,115.0,83.0,0,0,1,0,0,...,91,14,1,343.0,114.0,31.0,2.0,1.0,0.389920,0.000000
silveda01,1993,10900,144.336364,18032.374989,7.0,7.0,0,0,0,0,3,...,0,0,0,21.0,6.0,5.0,0.0,0.0,0.423077,42621.977247
valdese01,1989,68000,123.772727,131185.593293,19.0,1.0,19,0,0,0,0,...,0,0,0,1.0,1.0,0.0,0.0,0.0,1.000000,131185.593293
gideobr01,1987,62500,113.463636,131530.209191,29.0,0.0,29,0,0,0,0,...,0,0,0,1.0,1.0,0.0,1.0,0.0,1.000000,131530.209191
balleja01,1987,75000,113.463636,157836.251029,23.0,0.0,23,0,0,0,0,...,0,0,0,1.0,1.0,0.0,0.0,0.0,1.000000,157836.251029


In [12]:
#Let's filter out anyone with a salary under 10
#Players will need to have had at least 50 at bats  
#Later could do this and distinguish between position players and pitches
big_data = big_data[big_data['2016 Salary'] > 10]
big_data.head()

,,salary,CPI,2016 Salary,G_all,GS,G_p,G_c,G_1b,G_2b,G_3b,...,G_lf,G_cf,G_rf,AB,H,BB,HBP,SF,OBP,2016 Salary/OBP
playerID,yearID,,,,,,,,,,,,,,,,,,,,,
silveda01,1993,10900,144.336364,18032.374989,7.0,7.0,0,0,0,0,3,...,0,0,0,21.0,6.0,5.0,0.0,0.0,0.423077,42621.977247
valdese01,1989,68000,123.772727,131185.593293,19.0,1.0,19,0,0,0,0,...,0,0,0,1.0,1.0,0.0,0.0,0.0,1.000000,131185.593293
gideobr01,1987,62500,113.463636,131530.209191,29.0,0.0,29,0,0,0,0,...,0,0,0,1.0,1.0,0.0,1.0,0.0,1.000000,131530.209191
balleja01,1987,75000,113.463636,157836.251029,23.0,0.0,23,0,0,0,0,...,0,0,0,1.0,1.0,0.0,0.0,0.0,1.000000,157836.251029
byrdpa01,1995,109000,152.281818,170915.179746,17.0,0.0,17,0,0,0,0,...,0,0,0,1.0,1.0,0.0,0.0,0.0,1.000000,170915.179746


In [13]:
#Let's consider someone an eligible pitcher if they pitched over 10 games and had at least 50 at bats
pitchers = big_data[(big_data['G_p'] > 10) & (big_data['AB'] > 50)][:1]
#Let's consider someone an eligible position player if they had at least 200 at bats
position_players = big_data[big_data['AB'] > 200]

## All-Time Lineup
Notes:
Inflation is probably an unfair metric to use, as it seems that the Consumer Price Index has risen much slower than the salaries of MLB players.  

In [14]:
player_criteria = [big_data[(big_data['G_p'] > 10) & (big_data['AB'] > 50)][:1]]
for position in ['G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf']:
    player_criteria.append(position_players[position_players[position] > 50][:1])

all_time = pd.concat(player_criteria)
all_time[['salary', '2016 Salary', 'OBP', '2016 Salary/OBP']]

,,salary,2016 Salary,OBP,2016 Salary/OBP
playerID,yearID,,,,
moyerja01,1987,70000,147313.834294,0.294118,500867.036599
surhobj01,1987,62500,131530.209191,0.350000,375800.597688
millira01,1989,75000,144689.992603,0.394144,367099.181233
ripkebi01,1987,62500,131530.209191,0.363281,362061.651106
seitzke01,1987,62500,131530.209191,0.399171,329508.205724
reedjo01,1988,64500,130460.546289,0.380463,342899.679098
krukjo01,1986,60000,130807.055127,0.403077,324521.319972
youngge02,1987,62500,131530.209191,0.379538,346553.507694
felixju01,1989,68000,131185.593293,0.314978,416491.324162


In [15]:
print("""
Average salary: ${0:.2f}
Average salary adjusted for 2016: ${1:.2f}
Average OBP: {2:.3f}
""".format(all_time['salary'].mean(), all_time['2016 Salary'].mean(), all_time['OBP'].mean()))


Average salary: $65277.78
Average salary adjusted for 2016: $134508.65
Average OBP: 0.364

